## 1.1
##### Setting up necessary libraries

In [1]:
import gym
import keras 
import random
import math
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# training parameters
n_episodes = 1000
n_win_ticks = 195
max_env_steps = None

gamma = 1.0 # Discount factor, consideration of future rewards 
#(what reward to assign to a state that might be useful in the futur efor not now)
epsilon = 1.0 #controls exploration, 
# (an agent choosing an action that it believes has the best long term effect, with probabbility 1 - epsilon)
epsilon_min = 0.01 
epsilon_decay = 0.8 #how quickly it is going to stop exploring
alpha = 0.01 #learning rate
alpha_decay = 0.01

batch_size = 64
monitor = False
quiet = False

# environment parameters
memory = deque(maxlen=100000)
env = gym.make('CartPole-v0')
if max_env_steps is not None: env.max_episode_steps = max_env_steps

# building the neural network
# model definition
model = Sequential()
model.add(Dense(24, input_dim=4, activation ='relu'))
model.add(Dense(48, activation = 'relu'))
model.add(Dense(2, activation = 'relu'))
model.compile(loss = 'mse', optimizer = Adam(lr=alpha, decay=alpha_decay))

# defining necessary function

def remember(state, action, reward, next_state, done):
    memory.append((state, action, reward, next_state, done))
    
def choose_action(state, eplison):
    return env.action_space.sample() if (np.random.random() <= epsilon) else np.argmax(model.predict(state))

def get_epsilon(t):
    return max(epsilon_min, min(epsilon, 1.0 - math.log10((t+1)*epsilon_decay)))

def preprocess_state(state):
    return np.reshape(state, [1,4])

def replay(batch_size, epsilon):
    x_batch, y_batch = [], []
    minibatch = random.sample(memory, min(len(memory), batch_size))
    
    for state, action, reward, next_state, done in minibatch:
        y_target = model.predict(state)
        y_target[0][action] = reward if done else reward+gamma*np.max(model.predict(next_state))
        x_batch.append(state[0])
        y_batch.append(y_target[0])
        
    model.fit(np.array(x_batch), np.array(y_batch), batch_size = len(x_batch), verbose=0)
    
    if epsilon > epsilon_min:
        epsilon*=epsilon_decay
        
# define run function
def run():
    scores = deque(maxlen= 100)
    
    for e in range(n_episodes):
        state = preprocess_state(env.reset())
        done = False
        i=0
        while not done:
            action = choose_action(state, get_epsilon(e))
            next_state, reward, done, _ = env.step(action)
            env.render()
            next_state = preprocess_state(next_state)
            remember(state, action, reward, next_state, done)
            state = next_state
            i += 1
            
        scores.append(i)
        mean_score = np.mean(scores)
        
        if mean_score >= n_win_ticks and e >= 100:
            if not quiet: print('Ran {} episode. Solved after {} trials'.format(e, e-100))
            return e - 100
        if e%20==0 and not quiet:
            print('[Episode {}] - Mean survival time over last 100 episodes was {} ticks.'.format(e, mean_score))
            
        replay(batch_size, e)
        
    if not quiet: print('Didi not solve after {} episodes'.format(e))
    return e

run()

Using Theano backend.


[Episode 0] - Mean survival time over last 100 episodes was 41.0 ticks.
[Episode 20] - Mean survival time over last 100 episodes was 25.238095238095237 ticks.
[Episode 40] - Mean survival time over last 100 episodes was 25.24390243902439 ticks.
[Episode 60] - Mean survival time over last 100 episodes was 24.278688524590162 ticks.
[Episode 80] - Mean survival time over last 100 episodes was 23.45679012345679 ticks.
[Episode 100] - Mean survival time over last 100 episodes was 23.09 ticks.
[Episode 120] - Mean survival time over last 100 episodes was 22.28 ticks.
[Episode 140] - Mean survival time over last 100 episodes was 21.59 ticks.
[Episode 160] - Mean survival time over last 100 episodes was 22.07 ticks.
[Episode 180] - Mean survival time over last 100 episodes was 22.22 ticks.
[Episode 200] - Mean survival time over last 100 episodes was 22.36 ticks.
[Episode 220] - Mean survival time over last 100 episodes was 22.92 ticks.
[Episode 240] - Mean survival time over last 100 episodes

999